In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 67.5 MB/s eta 0:00:00


In [ ]:
import torch
from tqdm import tqdm
from transformers import GPTNeoForCausalLM, GPTNeoModel, GPT2Tokenizer, get_scheduler, pipeline
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import os
import json
import csv
import torch.nn as nn

In [ ]:
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer = GPT2Tokenizer.from_pretrained('EleutherAI/gpt-neo-125M', bos_token='<|startoftext|>', eos_token='<|endoftext|>')
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'deeplearning/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['GPTNeoMyself.ipynb', 'shortjokes.csv', 'openseminar_GAN.pptm', 'openseminar_GAN.pptm.gslides', 'Untitled0.ipynb', 'openseminar_GAN_final.docx', 'Davinci003.ipynb', 'Untitled', 'GPTNeo_RLHF.ipynb', 'GPTNEO.ipynb', 'baseline.ipynb', 'RM_Training.ipynb', 'trained_models']


In [ ]:
jokes = pd.read_csv(GOOGLE_DRIVE_PATH + 'shortjokes.csv')

In [ ]:
class JokesDataset(Dataset):
  def __init__(self):
    super().__init__()

    self.data_list = []
    self.end_of_text_token = "<|endoftext|>"
    
    # read from data source
    datas = pd.read_csv(GOOGLE_DRIVE_PATH + 'shortjokes.csv')
    
    x = 0
    for i in range(len(jokes.index)):
        data_str = f"JOKE:{jokes.iloc[i][1]}{self.end_of_text_token}"
        self.data_list.append(data_str)

  def __len__(self):
      return len(self.data_list)

  def __getitem__(self, index):
      text = self.data_list[index]
      inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=100, padding="max_length") #pad => special token X

      return inputs

In [ ]:
dataset = JokesDataset()
joke_loader = DataLoader(dataset, batch_size=16, shuffle=False)

In [ ]:
# default value settings
BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 3e-5
WARMUP_STEPS = 5000
MAX_SEQ_LEN = 400

In [ ]:
model = model.to(device)

In [ ]:
wte = model.transformer.wte
wpe = model.transformer.wpe
drop = model.transformer.drop
h = model.transformer.h

In [ ]:
attention_layers = h
layer=attention_layers[0]

In [ ]:
model.transformer.h = nn.ModuleList([layer])
model.train()

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_features=3

In [ ]:
from transformers import AdamW

# num of training steps = epoch * lengh of data
num_training_steps = EPOCHS*len(joke_loader)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
proc_seq_count = 0
sum_loss = 0.0
batch_count = 0

tmp_jokes_tens = None
models_folder = "trained_models" # where to save model

In [ ]:
# scheduler
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=WARMUP_STEPS,
    num_training_steps=num_training_steps,
)

In [ ]:
for epoch in range(EPOCHS):
  batch_count = 0
  print(f"EPOCH {epoch} started" + '=' * 30)
  
  # 그럼 이 for문은 몇번 도는거지?
  for idx, data in tqdm(enumerate(joke_loader)):
    # this data is returned from tokenizer => three types.
    input_ids, attention_mask = data['input_ids'].to(device), data['attention_mask'].to(device)
    # print("data 한개 길이 : ", len(input_ids), input_ids.shape, len(input_ids[0][0]))

    #forward process
    outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)

    # print(len(outputs))
    # print(outputs.keys())

    # what is loss and logits?
    loss, logits = outputs.loss, outputs.logits
    loss.backward()
    # way to copy.  .detach() , .data
    # print("cjhec", input_ids.shape, logits.shape)
    sum_loss = sum_loss + loss.detach().data

    # update optimizer for weight, scheduler for learning_rate
    optimizer.step()
    scheduler.step()

    # reset grads to 0 before another training
    optimizer.zero_grad()
    model.zero_grad()

    batch_count += 1
    if batch_count == 100:
        print(f"sum loss {sum_loss}")
        batch_count = 0
        sum_loss = 0.0

  # Store the model after each epoch to compare the performance of them
  torch.save(model.state_dict(), GOOGLE_DRIVE_PATH + models_folder + f"/gptneo_joker_{epoch}.pt")

EPOCH 0 started==============================


102it [00:17,  7.32it/s]

sum loss 3546.33642578125


202it [00:32,  7.17it/s]

sum loss 3447.1630859375


...

8302it [21:21,  6.83it/s]

sum loss 110.6781005859375


8402it [21:36,  6.86it/s]

sum loss 108.9552993774414


8472it [21:47,  6.47it/s]

## Load trained model

In [ ]:
MODEL_EPOCH = 3

models_folder = "/trained_models"

model_path = GOOGLE_DRIVE_PATH + models_folder + f"/gptneo_joker_{MODEL_EPOCH}.pt"
model.load_state_dict(torch.load(model_path)) # 여기서는 dict만 불러온다. 애초에 dict만 저장함.

jokes_output_file_path = f'generated_{MODEL_EPOCH}.jokes'

model.eval()

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_features=3

In [ ]:
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [ ]:
prompt = (
    "JOKE:"
)

tt = tokenizer(prompt, return_tensors="pt").to(device)

input_ids, attention_mask = tt['input_ids'], tt['attention_mask']

print(input_ids, attention_mask)

gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    attention_mask=attention_mask,
    max_length=100,
)

print(gen_tokens)

gen_text = tokenizer.batch_decode(gen_tokens)[0]

print(gen_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[45006,  7336,    25]], device='cuda:0') tensor([[1, 1, 1]], device='cuda:0')
tensor([[45006,  7336,    25,    32,  3516,   373,   523,  3735,   326,   345,
           460,  1239,  1265,   329,  8073,   366,  1639,  4398,   470,   587,
           422,   262,  9151,  1911,  3244,    11,   345,   821,  1016,   284,
           760,   703,   284,  2666,   340,   287,   262,  8848,   287,   257,
          7072,    13, 50256]], device='cuda:0')
JOKE:A guy was so fat that you can never ask for dinner "You haven't been from the ocean". Then, you're going to know how to leave it in the boat in a restaurant.<|endoftext|>


In [ ]:
cur_ids = torch.tensor(tokenizer.encode("JOKE:")).unsqueeze(0).to(device)
outputs = model(cur_ids, labels=cur_ids)
loss, logits = outputs[:2]

softmax_logits = torch.softmax(logits[0,-1], dim=0)

print(logits)

print(softmax_logits)

next_token_id = choose_from_top(softmax_logits.to('cpu').detach().numpy(), n=5)

print(next_token_id)

generated_word = tokenizer.decode(next_token_id)

print(generated_word)

tensor([[[ -1.9094,  -2.9512,  -6.0039,  ..., -12.3846,  -6.4041,   6.1725],
         [ -3.6738,  -5.3555,  -9.2521,  ..., -15.0279, -14.6613,  -4.0234],
         [ -4.6799,  -0.4297,  -1.7712,  ..., -10.5551,  -9.0834,  -4.3933]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)
tensor([6.2590e-06, 4.3888e-04, 1.1475e-04,  ..., 1.7578e-08, 7.6574e-08,
        8.3365e-06], device='cuda:0', grad_fn=<SoftmaxBackward0>)
5195
Why


In [ ]:
with torch.no_grad():
    for joke_idx in range(1000):
    
        joke_finished = False

        cur_ids = torch.tensor(tokenizer.encode("JOKE:")).unsqueeze(0).to(device)

        for i in range(100):
            outputs = model(cur_ids, labels=cur_ids)
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0,-1], dim=0)
            #Take the first(from only one in this case) batch and the last predicted embedding

            if i < 3:
                n = 20
            else:
                n = 3
            
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n) #Randomly(from the topN probability distribution) select the next word
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word to the running sequence

            print("토큰", next_token_id, torch.ones((1,1)).long().to(device) * next_token_id)

            if next_token_id in tokenizer.encode('<|endoftext|>'):
                joke_finished = True
                break

        
        if joke_finished:
            
            joke_num = joke_num + 1
            
            output_list = list(cur_ids.squeeze().to('cpu').numpy())
            output_text = tokenizer.decode(output_list)

            with open(jokes_output_file_path, 'a') as f:
                f.write(f"{output_text} \n\n")